# 0. Setup notebook

In [1]:
%pip install datasets
%pip uninstall -y pyarrow
%pip install pyarrow==15.0.2

Note: you may need to restart the kernel to use updated packages.
Found existing installation: pyarrow 15.0.2
Uninstalling pyarrow-15.0.2:
  Successfully uninstalled pyarrow-15.0.2
Note: you may need to restart the kernel to use updated packages.
  Using cached pyarrow-15.0.2-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-15.0.2-cp311-cp311-manylinux_2_28_x86_64.whl (38.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from datasets import load_dataset, load_dataset_builder
import subprocess

In [4]:
spark = (SparkSession.builder.master("yarn").appName("ALS-tune")
         .config("spark.driver.memory","8g")
         .config("spark.executor.memory","8g")
         .config("spark.driver.maxResultSize","2g")
         .getOrCreate())

25/05/05 22:04:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


----

# 1. Loading and preparing dataset

In [5]:
from datasets import load_dataset

dataset_user = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
print(dataset_user["full"][0])

README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

All_Beauty.jsonl:   0%|          | 0.00/327M [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

{'rating': 5.0, 'title': 'Such a lovely scent but not overpowering.', 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!", 'images': [], 'asin': 'B00YQ6X8EO', 'parent_asin': 'B00YQ6X8EO', 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ', 'timestamp': 1588687728923, 'helpful_vote': 0, 'verified_purchase': True}


In [6]:
dataset_item = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full")
print(dataset_item[0])

meta_All_Beauty.jsonl:   0%|          | 0.00/213M [00:00<?, ?B/s]

Generating full split:   0%|          | 0/112590 [00:00<?, ? examples/s]

{'main_category': 'All Beauty', 'title': 'Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)', 'average_rating': 4.8, 'rating_number': 10, 'features': [], 'description': [], 'price': 'None', 'images': {'hi_res': [None, 'https://m.media-amazon.com/images/I/71i77AuI9xL._SL1500_.jpg'], 'large': ['https://m.media-amazon.com/images/I/41qfjSfqNyL.jpg', 'https://m.media-amazon.com/images/I/41w2yznfuZL.jpg'], 'thumb': ['https://m.media-amazon.com/images/I/41qfjSfqNyL._SS40_.jpg', 'https://m.media-amazon.com/images/I/41w2yznfuZL._SS40_.jpg'], 'variant': ['MAIN', 'PT01']}, 'videos': {'title': [], 'url': [], 'user_id': []}, 'store': 'Howard Products', 'categories': [], 'details': '{"Package Dimensions": "7.1 x 5.5 x 3 inches; 2.38 Pounds", "UPC": "617390882781"}', 'parent_asin': 'B01CUPMQZE', 'bought_together': None, 'subtitle': None, 'author': None}


In [7]:
user_reviews = dataset_user
item_metadata = dataset_item

In [9]:
# For user_reviews, which is a DatasetDict
user_reviews["full"].to_json("user_reviews.jsonl")

# For item_metadata, which is a Dataset
item_metadata.to_json("item_metadata.jsonl")

Creating json from Arrow format:   0%|          | 0/702 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/113 [00:00<?, ?ba/s]

206804886

In [10]:
# push the data onto HDFS and check the folder 
!hadoop fs -put user_reviews.jsonl /user/hdfs/raw_review_All_Beauty.jsonl
!hadoop fs -put item_metadata.jsonl /user/hdfs/raw_meta_All_Beauty.jsonl
!hadoop fs -ls /user/hdfs

Found 2 items
-rw-r--r--   2 root hadoop  206804886 2025-05-05 22:05 /user/hdfs/raw_meta_All_Beauty.jsonl
-rw-r--r--   2 root hadoop  314275749 2025-05-05 22:05 /user/hdfs/raw_review_All_Beauty.jsonl


In [11]:
!hadoop fs -cat /user/hdfs/raw_review_All_Beauty.jsonl | head -n 5


{"rating":5.0,"title":"Such a lovely scent but not overpowering.","text":"This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!","images":[],"asin":"B00YQ6X8EO","parent_asin":"B00YQ6X8EO","user_id":"AGKHLEW2SOWHNMFQIJGBECAF7INQ","timestamp":1588687728923,"helpful_vote":0,"verified_purchase":true}
{"rating":4.0,"title":"Works great but smells a little weird.","text":"This product does what I need it to do, I just wish it was odorless or had a soft coconut smell. Having my head smell like an orange coffee is offputting. (granted, I did know the smell was described but I was hoping it would be light)","images":[],"asin":"B081TJ8YS3","parent_asin":"B081TJ8YS3","user_id":"AGKHLEW2SOWHNMFQIJGBECAF7INQ","timestamp":1588615855070,"helpful_vote":1,"ver

In [12]:
!hadoop fs -cat /user/hdfs/raw_meta_All_Beauty.jsonl | head -n 5

{"main_category":"All Beauty","title":"Howard LC0008 Leather Conditioner, 8-Ounce (4-Pack)","average_rating":4.8,"rating_number":10,"features":[],"description":[],"price":"None","images":{"hi_res":[null,"https:\/\/m.media-amazon.com\/images\/I\/71i77AuI9xL._SL1500_.jpg"],"large":["https:\/\/m.media-amazon.com\/images\/I\/41qfjSfqNyL.jpg","https:\/\/m.media-amazon.com\/images\/I\/41w2yznfuZL.jpg"],"thumb":["https:\/\/m.media-amazon.com\/images\/I\/41qfjSfqNyL._SS40_.jpg","https:\/\/m.media-amazon.com\/images\/I\/41w2yznfuZL._SS40_.jpg"],"variant":["MAIN","PT01"]},"videos":{"title":[],"url":[],"user_id":[]},"store":"Howard Products","categories":[],"details":"{\"Package Dimensions\": \"7.1 x 5.5 x 3 inches; 2.38 Pounds\", \"UPC\": \"617390882781\"}","parent_asin":"B01CUPMQZE","bought_together":null,"subtitle":null,"author":null}
{"main_category":"All Beauty","title":"Yes to Tomatoes Detoxifying Charcoal Cleanser (Pack of 2) with Charcoal Powder, Tomato Fruit Extract, and Gingko Biloba Le

---

# 2. ALS Model

## 2.1 Loading up data into ALS model

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, explode, split, regexp_replace
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

First of all we are loading the previously saved user and item review data from the HDFS in form of json file data into our dataframes. In order to get a first feel of how the data looks we are printing the schema as well as two first example rows of the respective dataframes.

In [3]:
# Loading user review dataframe + printing schema
df_user_review = spark.read.json("hdfs:///user/hdfs/raw_review_All_Beauty.jsonl")
print("User Review Schema:")
df_user_review.printSchema()
df_user_review.show(2)

# Loading item metadata dataframe + printing schema
df_item_metadata = spark.read.json("hdfs:///user/hdfs/raw_meta_All_Beauty.jsonl")
print("\nItem Metadata Schema:")
df_item_metadata.printSchema()
df_item_metadata.show(2)

User Review Schema:
root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)

+----------+------------+------+-----------+------+--------------------+-------------+--------------------+--------------------+-----------------+
|      asin|helpful_vote|images|parent_asin|rating|                text|    timestamp|               title|             user_id|verified_purchase|
+-----


Item Metadata Schema:
root
 |-- author: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- bought_together: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- images: struct (nullable = true)
 |    |-- hi_res: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- large: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- thumb: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- variant: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |-- main_category: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- price: string (nullable = 


[Stage 3:>                                                          (0 + 1) / 1]



After loading the data we are creating a specific dataframe where we are going to store specific user_ids and their according asin and ratings for the products. Then we need to select the necessary columns for the ALS model which are the user_id, item_id and ultimately the rating which is helpiing us to understand by how much a user is likely to buy an item according to similar users. The rating goes from 0-10. Our main dataframe will be the so called als_df which is represented by the A matrix in the report. Moreover we are printing some statistics to get a better overview of what data we have at hand inside matrix A.

In [6]:
# 1. DATA PREPARATION
# Select relevant columns for the recommendation system
ratings_df = df_user_review.select("user_id", "asin", "rating")

# Convert the user_id (string) to a numeric index for ALS
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.feature import StringIndexer

# Create numeric user IDs
user_indexer = StringIndexer(inputCol="user_id", outputCol="userId", handleInvalid="skip")
user_indexed_model = user_indexer.fit(ratings_df)
ratings_df = user_indexed_model.transform(ratings_df)

# Create numeric item IDs
item_indexer = StringIndexer(inputCol="asin", outputCol="itemId", handleInvalid="skip")
item_indexed_model = item_indexer.fit(ratings_df)
ratings_df = item_indexed_model.transform(ratings_df)

# Select necessary columns for ALS model
als_df = ratings_df.select("userId", "itemId", "rating")

# Verify data preparation
print("\nPrepared data sample:")
als_df.show(5)

# Check for any null values
print("\nNull values count:")
als_df.select([col(c).isNull().alias(c) for c in als_df.columns]).summary().show()

# Basic statistics for ratings
print("\nRatings statistics:")
als_df.describe("rating").show()

25/05/05 22:37:07 WARN DAGScheduler: Broadcasting large task binary with size 27.2 MiB
25/05/05 22:37:18 WARN DAGScheduler: Broadcasting large task binary with size 27.2 MiB



Prepared data sample:


25/05/05 22:37:22 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB


+-------+--------+------+
| userId|  itemId|rating|
+-------+--------+------+
|33934.0|   686.0|   5.0|
|33934.0|  9899.0|   4.0|
|71462.0| 14563.0|   5.0|
|26005.0|112872.0|   1.0|
|26005.0| 43503.0|   5.0|
+-------+--------+------+
only showing top 5 rows


Null values count:
+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    25%|
|    50%|
|    75%|
|    max|
+-------+


Ratings statistics:


25/05/05 22:37:26 WARN DAGScheduler: Broadcasting large task binary with size 30.9 MiB


+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            701528|
|   mean|3.9602453501499584|
| stddev|1.4944515968822232|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+



## 2.2 Splitting data

In order to train our ALS model and consequently evaluate it as well, we are splitting the A matrix into a training and testing part with a split of 80/20, where 80% of the data will be used for training the ALS model and 20% will not be shown to the model so that the weights for the X and Y matrix are not influenced by testing data. Therefore within the testing scenario the RMSE, root mean square error, will be assessed as the performance metric.

In [7]:
# 2. SPLIT DATA FOR TRAINING AND TESTING
# Split data into training and test sets
(training_data, test_data) = als_df.randomSplit([0.8, 0.2], seed=42)

# Cache the datasets for improved performance
training_data.cache()
test_data.cache()

print(f"Training data count: {training_data.count()}")
print(f"Test data count: {test_data.count()}")

25/05/05 22:37:34 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB
25/05/05 22:37:42 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB


Training data count: 561128


25/05/05 22:37:46 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB
25/05/05 22:37:53 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB


Test data count: 140400


Quickly we will check how man RDD partitions have been created to handle this dataset.

In [8]:
# Getting the number of current RDD partitions
print('Number of partitions: ', training_data.rdd.getNumPartitions())

# Requesting the number of worker nodes assigned to master node
print('Number of workers:', sc._conf.get('spark.executor.instances'))

Number of partitions:  3
Number of workers: 2


Due to multiple issues like crashing kernels (due to the size of the data) in GCP when training the ALS model, other solutions needed to be tested. We decided to increase the number of partitions which is creating a batched environment for the ALS training so the VMs are not training the whole dataset but smaller parts of it when running the RDDs into the ALS.

In [9]:
# Choosing a partition count suitable for large dataset size
training_data = training_data.repartition(200).persist()  
test_data = test_data.repartition(200).persist()

# Materializing the cache
training_data.count()
test_data.count()
print('New number of partitions for training data: ', training_data.rdd.getNumPartitions())
print('New number of partitions for testing data: ', test_data.rdd.getNumPartitions())

25/05/05 22:38:04 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB
25/05/05 22:38:09 WARN DAGScheduler: Broadcasting large task binary with size 20.1 MiB
25/05/05 22:38:48 WARN DAGScheduler: Broadcasting large task binary with size 20.1 MiB
25/05/05 22:39:23 WARN DAGScheduler: Broadcasting large task binary with size 34.4 MiB
25/05/05 22:39:26 WARN DAGScheduler: Broadcasting large task binary with size 20.1 MiB
25/05/05 22:39:57 WARN DAGScheduler: Broadcasting large task binary with size 20.1 MiB


New number of partitions for training data:  200
New number of partitions for testing data:  200


## 2.3 Training ALS

Ultimately we train the ALS using the library from pyspark, and we predefine arbitarily the hyperparameters. We chose a quite low amount of latent features and a low amount of maximum iterations due to the aforementioned high likelihood of the kernel crashing while training. In order ot keep the size as low as possible, we reduced the dimensionality of the matrices by reducing the latent features to 3 instead holding them at 5 or 10, since the matrices X and Y are having their dimension set by k = latent features.

In [11]:
# 3. TRAIN ALS MODEL
# Initialize ALS model
als = ALS(
    rank=3,                      # Number of latent factors
    maxIter=5,                   # Maximum number of iterations
    regParam=0.1,                # Regularization parameter
    userCol="userId",            # User column name
    itemCol="itemId",            # Item column name
    ratingCol="rating",          # Rating column name
    coldStartStrategy="drop"     # Strategy to handle cold start problem    
).setCheckpointInterval(2)       # cut lineage every 2 iterations

# Train the model
model = als.fit(training_data)

25/05/05 22:42:48 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:42:50 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:43:31 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:44:11 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:44:17 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:44:49 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:44:53 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:44:57 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:45:00 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:45:04 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:45:08 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:45:11 WARN DAGScheduler: Broadc

## 2.4 Evaluating ALS

After our first ALS model has been trained we are evaluating the ALS model with our testing set using the performance metric root mean squared error, which is common when it comes to continous optimization methods.

In [12]:
# 4. EVALUATE THE MODEL
# Make predictions on test data
predictions = model.transform(test_data)

# Evaluate model using RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating", 
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"\nRoot Mean Squared Error (RMSE): {rmse}")

25/05/05 22:45:41 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:45:42 WARN DAGScheduler: Broadcasting large task binary with size 20.2 MiB
25/05/05 22:45:43 WARN DAGScheduler: Broadcasting large task binary with size 20.1 MiB
25/05/05 22:46:26 WARN DAGScheduler: Broadcasting large task binary with size 20.3 MiB



Root Mean Squared Error (RMSE): 5.0500559582618205


Here we are saving the ALS model for HDFS, however this can be only used when staying within the same cluster and not spinning up a new cluster to tetry training of the notebook and ALS.

In [ ]:
# SAVING THE FIRST MODEL AND USER/ITEM MAPPINGS

# Define location where model will be saved
hdfs_path = "hdfs:///st446-cluster-w09-m"

# Save the first model due to multiple kernel crashes occuring during running/training
model.save(hdfs_path + "amazon_first_als_model")

# Save user and item mappings for future use
ratings_df.select("user_id", "userId").distinct().write.parquet(hdfs_path + "user_mapping")
ratings_df.select("asin", "itemId").distinct().write.parquet(hdfs_path + "item_mapping")

print("\nModel and mappings saved successfully")

In [ ]:
from pyspark.ml.recommendation import ALSModel

hdfs_model_path = "hdfs:///user/st446-cluster-w09-m/amazon_first_als_model"
loaded_model = ALSModel.load(hdfs_model_path)

user_map = spark.read.parquet("hdfs:///user/{YOUR_LINUX_USERNAME}/user_mapping")
item_map = spark.read.parquet("hdfs:///user/{YOUR_LINUX_USERNAME}/item_mapping")


From here on we will focus on the main notebook "ALS Model Local.ipynb" as it is the notebook which was able to run and train the ALS reasonably without too many kernel crashes.